In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
from pathlib import Path
from datetime import datetime
import warnings
import re
warnings.filterwarnings('ignore')

EXCEL_PATH = r"\assessment_of_wells_chile\data\DGA\DGA_consolidado_nacional\outputs_v2\Registros_en_Chile_arreglado_v2.xlsx"
OUTPUT_FOLDER = r"\assessment_of_wells_chile\data\DGA\DGA_dataset_analysis_output"

GDB_PATH = r"\assessment_of_wells_chile\arcgis\assessment_of_wells_chile\Default.gdb"

REFERENCE_LAYERS = [
    {
        'path': GDB_PATH,
        'layer_name': 'CHL_Municipalities',
        'prefix': 'Muni',
        'name_col': 'NAME',
        'code_col': 'Code_Muni',
        'native_crs': 'EPSG:3857',
        'is_gdb': True
    },
    {
        'path': GDB_PATH,
        'layer_name': 'CHL_Regions',
        'prefix': 'Region',
        'name_col': 'NAME',
        'code_col': 'ID',
        'native_crs': 'EPSG:3857',
        'is_gdb': True
    },
    {
        'path': r"\assessment_of_wells_chile\data\Basins\Cuencas_BNA\Cuencas_BNA.shp",
        'layer_name': None,
        'prefix': 'Cuenca',
        'name_col': 'NOM_CUEN',
        'code_col': 'COD_CUEN',
        'native_crs': 'EPSG:32719',
        'is_gdb': False
    },
    {
        'path': r"\assessment_of_wells_chile\data\Aquifers\INV_ACUIFEROS_SHAC_202302\INV_ACUIFEROS_SHAC.shp",
        'layer_name': None,
        'prefix': 'SHAC',
        'name_col': 'SHAC',
        'code_col': 'COD_SHAC',
        'native_crs': 'EPSG:32719',
        'is_gdb': False
    }
]

TARGET_CRS = "EPSG:4326"

DATE_COL_PRIMARY = 'Fecha Toma Razón'
DATE_COL_SECONDARY = 'Fecha de Resolución/ Envío al Juez/ Inscripción C.B.R.'

UNIT_CONVERSION = {
    'Lt/s': 1.0,
    'Lt/min': 1.0 / 60.0,
    'm3/h': 1000.0 / 3600.0,
    'm3/año': 1000.0 / (365.25 * 24 * 3600),
    'Lt/h': 1.0 / 3600.0,
    'lt/día': 1.0 / 86400.0,
    'm3/dia': 1000.0 / 86400.0,
    'm3/s': 1000.0,
    'm3/mes': 1000.0 / (30.0 * 24 * 3600)
}

EXCLUDED_UNITS = ['%', 'Acciones']

MAX_REALISTIC_FLOW_LS = 500.0
MIN_REALISTIC_FLOW_LS = 0.0

def create_output_folder(path):
    Path(path).mkdir(parents=True, exist_ok=True)
    print(f"Output folder ready: {path}")

def load_reference_layer(layer_config):
    path = layer_config['path']
    prefix = layer_config['prefix']
    native_crs = layer_config['native_crs']
    is_gdb = layer_config['is_gdb']
    layer_name = layer_config.get('layer_name')
    name_col = layer_config['name_col']
    code_col = layer_config['code_col']
    
    print(f"\n  Loading {prefix} layer...")
    print(f"    Path: {path}")
    
    try:
        if is_gdb:
            print(f"    Layer in GDB: {layer_name}")
            gdf = gpd.read_file(path, layer=layer_name)
        else:
            gdf = gpd.read_file(path)
        
        print(f"    Records loaded: {len(gdf):,}")
        print(f"    Native CRS: {gdf.crs}")
        
        if gdf.crs is None:
            print(f"    WARNING: No CRS defined, assuming {native_crs}")
            gdf = gdf.set_crs(native_crs)
        
        if gdf.crs.to_string() != TARGET_CRS:
            print(f"    Reprojecting from {gdf.crs} to {TARGET_CRS}...")
            gdf = gdf.to_crs(TARGET_CRS)
        
        available_cols = gdf.columns.tolist()
        print(f"    Available columns: {available_cols}")
        
        if name_col not in available_cols:
            print(f"    WARNING: Name column '{name_col}' not found!")
            for col in available_cols:
                if 'name' in col.lower() or 'nom' in col.lower():
                    print(f"    Using '{col}' as name column instead")
                    name_col = col
                    break
        
        if code_col not in available_cols:
            print(f"    WARNING: Code column '{code_col}' not found!")
            for col in available_cols:
                if 'cod' in col.lower() or 'id' in col.lower():
                    print(f"    Using '{col}' as code column instead")
                    code_col = col
                    break
        
        cols_to_keep = ['geometry']
        if name_col in gdf.columns:
            cols_to_keep.append(name_col)
        if code_col in gdf.columns and code_col != name_col:
            cols_to_keep.append(code_col)
        
        gdf = gdf[cols_to_keep].copy()
        
        rename_dict = {}
        if name_col in gdf.columns:
            rename_dict[name_col] = f'{prefix}_Name'
        if code_col in gdf.columns and code_col != name_col:
            rename_dict[code_col] = f'{prefix}_Code'
        
        gdf = gdf.rename(columns=rename_dict)
        
        print(f"    Final columns: {gdf.columns.tolist()}")
        print(f"    Final CRS: {gdf.crs}")
        
        return gdf, prefix
        
    except Exception as e:
        print(f"    ERROR loading {prefix}: {str(e)}")
        return None, prefix

def load_excel_data(excel_path):
    print(f"\nLoading Excel file: {excel_path}")
    df = pd.read_excel(excel_path)
    print(f"  -> Loaded {len(df):,} records")
    return df

def convert_flow_to_ls(row):
    flow = row['Caudal Anual Prom']
    unit = row['Unidad de Caudal']

    if pd.isna(flow) or pd.isna(unit):
        return np.nan

    if unit in EXCLUDED_UNITS:
        return np.nan

    unit_clean = str(unit).strip()
    if unit_clean in UNIT_CONVERSION:
        return float(flow) * UNIT_CONVERSION[unit_clean]
    else:
        return np.nan

def classify_anomaly(row):
    flow_ls = row['Caudal_Ls']
    unit = row['Unidad de Caudal']

    if pd.isna(flow_ls):
        if pd.isna(unit):
            return 'Missing unit'
        if str(unit).strip() in EXCLUDED_UNITS:
            return 'Non-volumetric unit (excluded)'
        return 'Unconvertible unit'

    if flow_ls < MIN_REALISTIC_FLOW_LS:
        return 'Negative value'

    if flow_ls > MAX_REALISTIC_FLOW_LS:
        return 'Unrealistically high value'

    return 'Valid'

def get_anomaly_severity(row):
    status = row['Anomaly_Status']
    flow_ls = row['Caudal_Ls']

    if status == 'Valid':
        return 'None'
    elif status == 'Negative value':
        return 'High - Data Entry Error'
    elif status == 'Unrealistically high value':
        if flow_ls > 10000:
            return 'Critical - Possible Unit Confusion'
        elif flow_ls > 1000:
            return 'High - Review Required'
        else:
            return 'Medium - Verify Against Resolution'
    else:
        return 'Low - Missing Data'

def create_geodataframe(df, lat_col='lat_wgs84', lon_col='lon_wgs84'):
    mask = (
        df[lat_col].notna() &
        df[lon_col].notna() &
        (df[lat_col] != 0) &
        (df[lon_col] != 0) &
        (df[lat_col] >= -56) & (df[lat_col] <= -17) &
        (df[lon_col] >= -76) & (df[lon_col] <= -66)
    )
    
    if 'Coord_Valida_Chile_BBox' in df.columns:
        mask = mask & (df['Coord_Valida_Chile_BBox'] == True)

    df_spatial = df[mask].copy()

    if len(df_spatial) == 0:
        print("  WARNING: No valid coordinates found!")
        return None

    print(f"  Creating GeoDataFrame with {len(df_spatial):,} valid points...")
    
    gdf = gpd.GeoDataFrame(
        df_spatial,
        geometry=gpd.points_from_xy(df_spatial[lon_col], df_spatial[lat_col]),
        crs=TARGET_CRS
    )

    return gdf

def perform_spatial_join(gdf_points, gdf_polygons, prefix):
    if gdf_points is None or len(gdf_points) == 0:
        print(f"  WARNING: No points to join for {prefix}")
        return gdf_points
    
    if gdf_polygons is None or len(gdf_polygons) == 0:
        print(f"  WARNING: No polygons available for {prefix}")
        return gdf_points

    print(f"  Performing spatial join with {prefix}...")
    print(f"    Points CRS: {gdf_points.crs}")
    print(f"    Polygons CRS: {gdf_polygons.crs}")
    
    if gdf_points.crs != gdf_polygons.crs:
        print(f"    Reprojecting polygons to match points CRS...")
        gdf_polygons = gdf_polygons.to_crs(gdf_points.crs)

    cols_to_drop = [col for col in gdf_points.columns if col.startswith('index_')]
    if cols_to_drop:
        gdf_points = gdf_points.drop(columns=cols_to_drop)

    gdf_points = gdf_points.reset_index(drop=True)
    gdf_polygons = gdf_polygons.reset_index(drop=True)

    try:
        gdf_joined = gpd.sjoin(
            gdf_points, 
            gdf_polygons, 
            how='left', 
            predicate='within'
        )
        
        cols_to_drop = [col for col in gdf_joined.columns if col.startswith('index_')]
        if cols_to_drop:
            gdf_joined = gdf_joined.drop(columns=cols_to_drop)

        name_col = f'{prefix}_Name'
        if name_col in gdf_joined.columns:
            n_matched = gdf_joined[name_col].notna().sum()
            print(f"    Matched {n_matched:,} of {len(gdf_joined):,} points ({100*n_matched/len(gdf_joined):.1f}%)")
        
        gdf_joined = gdf_joined.reset_index(drop=True)
        
        return gdf_joined
        
    except Exception as e:
        print(f"    ERROR in spatial join: {str(e)}")
        return gdf_points

def calculate_statistics(gdf, groupby_col, flow_col='Caudal_Ls'):
    if groupby_col not in gdf.columns:
        print(f"  WARNING: Column '{groupby_col}' not found in data")
        return pd.DataFrame()
    
    gdf_filtered = gdf[gdf[groupby_col].notna()].copy()

    if len(gdf_filtered) == 0:
        print(f"  WARNING: No valid data for grouping by '{groupby_col}'")
        return pd.DataFrame()

    df_filtered = pd.DataFrame(gdf_filtered.drop(columns='geometry'))

    stats = df_filtered.groupby(groupby_col).agg(
        N_Permits=(flow_col, 'count'),
        Total_Flow_Ls=(flow_col, 'sum'),
        Mean_Flow_Ls=(flow_col, 'mean'),
        Median_Flow_Ls=(flow_col, 'median'),
        Std_Flow_Ls=(flow_col, 'std'),
        Min_Flow_Ls=(flow_col, 'min'),
        Max_Flow_Ls=(flow_col, 'max'),
        Q25_Flow_Ls=(flow_col, lambda x: x.quantile(0.25)),
        Q75_Flow_Ls=(flow_col, lambda x: x.quantile(0.75))
    ).reset_index()

    stats = stats.sort_values('Total_Flow_Ls', ascending=False)

    numeric_cols = ['Total_Flow_Ls', 'Mean_Flow_Ls', 'Median_Flow_Ls', 
                    'Std_Flow_Ls', 'Min_Flow_Ls', 'Max_Flow_Ls', 
                    'Q25_Flow_Ls', 'Q75_Flow_Ls']
    for col in numeric_cols:
        if col in stats.columns:
            stats[col] = stats[col].round(2)

    return stats

def parse_chilean_date(date_str):
    if pd.isna(date_str) or str(date_str).strip() == '':
        return None, False

    date_str = str(date_str).strip()

    formats_to_try = [
        '%d-%m-%Y',
        '%d/%m/%Y',
        '%d-%m-%y',
        '%d/%m/%y',
        '%Y-%m-%d',
        '%d.%m.%Y',
    ]

    for fmt in formats_to_try:
        try:
            parsed_date = datetime.strptime(date_str, fmt)
            if 1900 <= parsed_date.year <= 2100:
                return parsed_date, True
        except ValueError:
            continue

    year_match = re.search(r'(\d{4})', date_str)
    if year_match:
        year = int(year_match.group(1))
        if 1900 <= year <= 2100:
            return datetime(year, 1, 1), True

    return None, False

def convert_to_american_format(dt):
    if dt is None:
        return None
    return dt.strftime('%m-%d-%Y')

def extract_year(dt):
    if dt is None:
        return None
    return dt.year

def process_dates(df, primary_col, secondary_col):
    print("\n" + "="*60)
    print("PROCESSING DATES (Chilean Format)")
    print("="*60)

    total_records = len(df)

    df['Fecha_Parsed'] = None
    df['Fecha_Formato_US'] = None
    df['Ano'] = None
    df['Fecha_Source'] = None

    stats = {
        'total_records': total_records,
        'primary_col_present': 0,
        'primary_col_parsed': 0,
        'secondary_col_present': 0,
        'secondary_col_parsed': 0,
        'total_with_date': 0,
        'total_without_date': 0,
        'recovered_from_secondary': 0
    }

    primary_exists = primary_col in df.columns
    secondary_exists = secondary_col in df.columns

    if primary_exists:
        print(f"Primary date column found: '{primary_col}'")
        stats['primary_col_present'] = df[primary_col].notna().sum()
        print(f"  Records with non-empty primary date: {stats['primary_col_present']:,}")
    else:
        print(f"WARNING: Primary date column '{primary_col}' NOT FOUND")

    if secondary_exists:
        print(f"Secondary date column found: '{secondary_col}'")
        stats['secondary_col_present'] = df[secondary_col].notna().sum()
        print(f"  Records with non-empty secondary date: {stats['secondary_col_present']:,}")

    print("\nParsing dates...")

    for idx in df.index:
        parsed_date = None
        source = None
        
        if primary_exists and pd.notna(df.at[idx, primary_col]):
            parsed_date, success = parse_chilean_date(df.at[idx, primary_col])
            if success:
                source = 'Fecha Toma Razón'
                stats['primary_col_parsed'] += 1
        
        if parsed_date is None and secondary_exists and pd.notna(df.at[idx, secondary_col]):
            parsed_date, success = parse_chilean_date(df.at[idx, secondary_col])
            if success:
                source = 'Fecha Resolución/Envío'
                stats['secondary_col_parsed'] += 1
                stats['recovered_from_secondary'] += 1
        
        if parsed_date is not None:
            df.at[idx, 'Fecha_Parsed'] = parsed_date
            df.at[idx, 'Fecha_Formato_US'] = convert_to_american_format(parsed_date)
            df.at[idx, 'Ano'] = extract_year(parsed_date)
            df.at[idx, 'Fecha_Source'] = source
            stats['total_with_date'] += 1
        else:
            stats['total_without_date'] += 1

    df['Ano'] = pd.to_numeric(df['Ano'], errors='coerce')
    df.loc[df['Ano'].notna(), 'Ano'] = df.loc[df['Ano'].notna(), 'Ano'].astype(int)

    print("\n" + "-"*50)
    print("DATE PROCESSING SUMMARY")
    print("-"*50)
    print(f"Total records in dataset: {stats['total_records']:,}")
    print(f"\nPrimary column ('{primary_col}'):")
    print(f"  - Records with data: {stats['primary_col_present']:,} ({100*stats['primary_col_present']/total_records:.1f}%)")
    print(f"  - Successfully parsed: {stats['primary_col_parsed']:,} ({100*stats['primary_col_parsed']/total_records:.1f}%)")

    if secondary_exists:
        print(f"\nSecondary column ('{secondary_col}'):")
        print(f"  - Records with data: {stats['secondary_col_present']:,} ({100*stats['secondary_col_present']/total_records:.1f}%)")
        print(f"  - Used as fallback: {stats['recovered_from_secondary']:,} ({100*stats['recovered_from_secondary']/total_records:.1f}%)")

    print(f"\nFINAL RESULTS:")
    print(f"  ✓ Records WITH valid date: {stats['total_with_date']:,} ({100*stats['total_with_date']/total_records:.1f}%)")
    print(f"  ✗ Records WITHOUT valid date: {stats['total_without_date']:,} ({100*stats['total_without_date']/total_records:.1f}%)")

    valid_years = df['Ano'].dropna()
    if len(valid_years) > 0:
        print(f"\nYear range: {int(valid_years.min())} - {int(valid_years.max())}")

    if df['Fecha_Source'].notna().any():
        print("\nDate source distribution:")
        source_counts = df['Fecha_Source'].value_counts()
        for source, count in source_counts.items():
            print(f"  {source}: {count:,} ({100*count/stats['total_with_date']:.1f}%)")

    return df, stats

def perform_temporal_analysis(gdf_valid, date_quality_stats):
    print("\n" + "="*60)
    print("TEMPORAL ANALYSIS")
    print("="*60)

    df_temp = pd.DataFrame(gdf_valid.drop(columns='geometry'))
    df_temp = df_temp[df_temp['Ano'].notna()].copy()
    df_temp['Year'] = df_temp['Ano'].astype(int)

    current_year = datetime.now().year
    df_temp = df_temp[(df_temp['Year'] >= 1900) & (df_temp['Year'] <= current_year)]

    if len(df_temp) == 0:
        print("WARNING: No valid dates found for temporal analysis")
        return None, "No valid dates found for temporal analysis."

    print(f"Records with valid dates for analysis: {len(df_temp):,}")
    print(f"Year range: {df_temp['Year'].min()} - {df_temp['Year'].max()}")

    yearly_stats = df_temp.groupby('Year').agg(
        N_Permits=('Caudal_Ls', 'count'),
        Total_Flow_Ls=('Caudal_Ls', 'sum'),
        Mean_Flow_Ls=('Caudal_Ls', 'mean'),
        Median_Flow_Ls=('Caudal_Ls', 'median'),
        Max_Flow_Ls=('Caudal_Ls', 'max'),
        Min_Flow_Ls=('Caudal_Ls', 'min'),
        Std_Flow_Ls=('Caudal_Ls', 'std'),
        Q25_Flow_Ls=('Caudal_Ls', lambda x: x.quantile(0.25)),
        Q75_Flow_Ls=('Caudal_Ls', lambda x: x.quantile(0.75))
    ).reset_index()

    yearly_stats = yearly_stats.sort_values('Year')
    yearly_stats['Cumulative_Permits'] = yearly_stats['N_Permits'].cumsum()
    yearly_stats['Cumulative_Flow_Ls'] = yearly_stats['Total_Flow_Ls'].cumsum()
    yearly_stats['YoY_Permits_Change'] = yearly_stats['N_Permits'].diff()
    yearly_stats['YoY_Permits_Pct_Change'] = yearly_stats['N_Permits'].pct_change() * 100
    yearly_stats['YoY_Flow_Change'] = yearly_stats['Total_Flow_Ls'].diff()
    yearly_stats['YoY_Flow_Pct_Change'] = yearly_stats['Total_Flow_Ls'].pct_change() * 100
    yearly_stats['MA5_Permits'] = yearly_stats['N_Permits'].rolling(window=5, center=True, min_periods=1).mean()
    yearly_stats['MA5_Flow'] = yearly_stats['Total_Flow_Ls'].rolling(window=5, center=True, min_periods=1).mean()

    df_temp['Decade'] = (df_temp['Year'] // 10) * 10
    decade_stats = df_temp.groupby('Decade').agg(
        N_Permits=('Caudal_Ls', 'count'),
        Total_Flow_Ls=('Caudal_Ls', 'sum'),
        Mean_Flow_Ls=('Caudal_Ls', 'mean'),
        Median_Flow_Ls=('Caudal_Ls', 'median'),
        Max_Flow_Ls=('Caudal_Ls', 'max'),
        Std_Flow_Ls=('Caudal_Ls', 'std')
    ).reset_index()
    decade_stats['Decade_Label'] = decade_stats['Decade'].astype(str) + 's'
    decade_stats = decade_stats.sort_values('Decade')

    df_temp['Period_5yr'] = (df_temp['Year'] // 5) * 5
    period5_stats = df_temp.groupby('Period_5yr').agg(
        N_Permits=('Caudal_Ls', 'count'),
        Total_Flow_Ls=('Caudal_Ls', 'sum'),
        Mean_Flow_Ls=('Caudal_Ls', 'mean'),
        Median_Flow_Ls=('Caudal_Ls', 'median'),
        Max_Flow_Ls=('Caudal_Ls', 'max')
    ).reset_index()
    period5_stats['Period_Label'] = period5_stats['Period_5yr'].astype(str) + '-' + (period5_stats['Period_5yr'] + 4).astype(str)
    period5_stats = period5_stats.sort_values('Period_5yr')

    peak_permits_year = yearly_stats.loc[yearly_stats['N_Permits'].idxmax()]
    peak_flow_year = yearly_stats.loc[yearly_stats['Total_Flow_Ls'].idxmax()]

    significant_years = yearly_stats[yearly_stats['N_Permits'] >= 10]
    peak_mean_flow_year = None
    if len(significant_years) > 0:
        peak_mean_flow_year = significant_years.loc[significant_years['Mean_Flow_Ls'].idxmax()]

    top10_permits_years = yearly_stats.nlargest(10, 'N_Permits')
    top10_flow_years = yearly_stats.nlargest(10, 'Total_Flow_Ls')
    
    if len(significant_years) >= 10:
        top10_mean_flow_years = significant_years.nlargest(10, 'Mean_Flow_Ls')
    else:
        top10_mean_flow_years = significant_years.nlargest(len(significant_years), 'Mean_Flow_Ls')

    p75_permits = yearly_stats['N_Permits'].quantile(0.75)
    p75_flow = yearly_stats['Total_Flow_Ls'].quantile(0.75)
    p90_permits = yearly_stats['N_Permits'].quantile(0.90)
    p90_flow = yearly_stats['Total_Flow_Ls'].quantile(0.90)

    yearly_stats['High_Permits'] = yearly_stats['N_Permits'] > p75_permits
    yearly_stats['High_Flow'] = yearly_stats['Total_Flow_Ls'] > p75_flow

    high_permit_years = yearly_stats[yearly_stats['High_Permits']]['Year'].tolist()
    high_flow_years = yearly_stats[yearly_stats['High_Flow']]['Year'].tolist()

    correlation_permits_flow = yearly_stats['N_Permits'].corr(yearly_stats['Total_Flow_Ls'])
    correlation_permits_mean_flow = yearly_stats['N_Permits'].corr(yearly_stats['Mean_Flow_Ls'])

    trend_info = None
    recent_years = yearly_stats[yearly_stats['Year'] >= 1990].copy()
    if len(recent_years) >= 5:
        try:
            from scipy import stats as scipy_stats
            x = recent_years['Year'].values
            y_permits = recent_years['N_Permits'].values
            y_flow = recent_years['Total_Flow_Ls'].values
            
            slope_permits, intercept_permits, r_permits, p_permits, se_permits = scipy_stats.linregress(x, y_permits)
            slope_flow, intercept_flow, r_flow, p_flow, se_flow = scipy_stats.linregress(x, y_flow)
            
            trend_info = {
                'permits_slope': slope_permits,
                'permits_r_squared': r_permits**2,
                'permits_p_value': p_permits,
                'flow_slope': slope_flow,
                'flow_r_squared': r_flow**2,
                'flow_p_value': p_flow
            }
        except ImportError:
            print("  scipy not available for trend analysis")

    regional_yearly = None
    regional_peaks = None

    if 'Region_Name' in df_temp.columns:
        regional_yearly = df_temp.groupby(['Year', 'Region_Name']).agg(
            N_Permits=('Caudal_Ls', 'count'),
            Total_Flow_Ls=('Caudal_Ls', 'sum'),
            Mean_Flow_Ls=('Caudal_Ls', 'mean')
        ).reset_index()
        
        regional_peaks = regional_yearly.loc[
            regional_yearly.groupby('Region_Name')['N_Permits'].idxmax()
        ][['Region_Name', 'Year', 'N_Permits', 'Total_Flow_Ls']].rename(
            columns={'Year': 'Peak_Year', 'N_Permits': 'Peak_Permits', 'Total_Flow_Ls': 'Peak_Flow_Ls'}
        )

    def find_consecutive_periods(years_list, min_gap=2):
        if len(years_list) == 0:
            return []
        years_sorted = sorted(years_list)
        periods = []
        period_start = years_sorted[0]
        period_end = years_sorted[0]
        for year in years_sorted[1:]:
            if year - period_end <= min_gap:
                period_end = year
            else:
                periods.append((period_start, period_end))
                period_start = year
                period_end = year
        periods.append((period_start, period_end))
        return periods

    high_permit_periods = find_consecutive_periods(high_permit_years)
    high_flow_periods = find_consecutive_periods(high_flow_years)

    temporal_stats = {
        'yearly': yearly_stats,
        'decade': decade_stats,
        'period_5yr': period5_stats,
        'peak_permits_year': peak_permits_year,
        'peak_flow_year': peak_flow_year,
        'peak_mean_flow_year': peak_mean_flow_year,
        'top10_permits_years': top10_permits_years,
        'top10_flow_years': top10_flow_years,
        'top10_mean_flow_years': top10_mean_flow_years,
        'correlation_permits_flow': correlation_permits_flow,
        'correlation_permits_mean_flow': correlation_permits_mean_flow,
        'trend_info': trend_info,
        'regional_yearly': regional_yearly,
        'regional_peaks': regional_peaks,
        'date_quality_stats': date_quality_stats,
        'total_with_dates': len(df_temp),
        'year_range': (df_temp['Year'].min(), df_temp['Year'].max()),
        'p75_permits': p75_permits,
        'p75_flow': p75_flow,
        'p90_permits': p90_permits,
        'p90_flow': p90_flow,
        'high_permit_years': high_permit_years,
        'high_flow_years': high_flow_years,
        'high_permit_periods': high_permit_periods,
        'high_flow_periods': high_flow_periods
    }

    return temporal_stats, None

def generate_temporal_summary(temporal_stats):
    if temporal_stats is None:
        return "\nTEMPORAL ANALYSIS: Not available\n"

    lines = []
    lines.append("\n" + "="*80)
    lines.append("TEMPORAL ANALYSIS OF WATER RIGHTS ALLOCATION IN CHILE")
    lines.append("="*80)

    dq = temporal_stats['date_quality_stats']
    lines.append("\n" + "-"*60)
    lines.append("DATE DATA QUALITY REPORT")
    lines.append("-"*60)
    lines.append(f"Total records in dataset: {dq['total_records']:,}")
    lines.append(f"Records with valid dates: {dq['total_with_date']:,} ({100*dq['total_with_date']/dq['total_records']:.1f}%)")
    lines.append(f"Records without valid dates: {dq['total_without_date']:,} ({100*dq['total_without_date']/dq['total_records']:.1f}%)")

    lines.append("\n" + "-"*60)
    lines.append("PEAK YEARS")
    lines.append("-"*60)
    
    peak_permits = temporal_stats['peak_permits_year']
    lines.append(f"\nPeak year for permits: {int(peak_permits['Year'])} ({int(peak_permits['N_Permits']):,} permits)")
    
    peak_flow = temporal_stats['peak_flow_year']
    lines.append(f"Peak year for total flow: {int(peak_flow['Year'])} ({peak_flow['Total_Flow_Ls']:,.2f} L/s)")

    lines.append("\n" + "-"*60)
    lines.append("TOP 10 YEARS BY PERMITS")
    lines.append("-"*60)
    for rank, (idx, row) in enumerate(temporal_stats['top10_permits_years'].iterrows(), 1):
        lines.append(f"  {rank}. {int(row['Year'])}: {int(row['N_Permits']):,} permits, {row['Total_Flow_Ls']:,.2f} L/s")

    lines.append("\n" + "-"*60)
    lines.append("DECADE ANALYSIS")
    lines.append("-"*60)
    for idx, row in temporal_stats['decade'].iterrows():
        lines.append(f"  {row['Decade_Label']}: {int(row['N_Permits']):,} permits, {row['Total_Flow_Ls']:,.2f} L/s total")

    return "\n".join(lines)

def generate_analysis_summary(gdf_valid, gdf_anomalies, stats_dict, temporal_stats=None):
    
    df_valid = pd.DataFrame(gdf_valid.drop(columns='geometry')) if gdf_valid is not None else pd.DataFrame()
    df_anomalies = pd.DataFrame(gdf_anomalies.drop(columns='geometry')) if gdf_anomalies is not None else pd.DataFrame()

    summary_lines = []
    summary_lines.append("="*80)
    summary_lines.append("DGA GROUNDWATER RIGHTS DATABASE ANALYSIS SUMMARY")
    summary_lines.append("(All spatial assignments from shapefiles via spatial joins)")
    summary_lines.append(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    summary_lines.append("="*80)

    summary_lines.append("\n" + "="*60)
    summary_lines.append("1. DATASET OVERVIEW")
    summary_lines.append("="*60)
    summary_lines.append(f"Valid records for analysis: {len(df_valid):,}")
    summary_lines.append(f"Anomalous records: {len(df_anomalies):,}")

    if len(df_valid) > 0:
        summary_lines.append("\n--- Flow Rate Statistics (Valid Records Only, L/s) ---")
        summary_lines.append(f"  Total allocated flow: {df_valid['Caudal_Ls'].sum():,.2f} L/s")
        summary_lines.append(f"  Mean flow per permit: {df_valid['Caudal_Ls'].mean():.2f} L/s")
        summary_lines.append(f"  Median flow per permit: {df_valid['Caudal_Ls'].median():.2f} L/s")

    for scale_name, stats_df in stats_dict.items():
        summary_lines.append("\n" + "="*60)
        summary_lines.append(f"{scale_name.upper()} ANALYSIS (from shapefile spatial join)")
        summary_lines.append("="*60)
        if len(stats_df) > 0:
            summary_lines.append(f"Number of {scale_name} units with data: {len(stats_df)}")
            summary_lines.append(f"\nTop 10 by Total Allocated Flow:")
            for idx, row in stats_df.head(10).iterrows():
                name_col = [c for c in row.index if '_Name' in c or c == scale_name][0] if any('_Name' in c or c == scale_name for c in row.index) else row.index[0]
                summary_lines.append(f"  {row[name_col]}: {row['Total_Flow_Ls']:,.2f} L/s ({row['N_Permits']:,} permits)")
        else:
            summary_lines.append("  No data available")

    if temporal_stats is not None:
        summary_lines.append(generate_temporal_summary(temporal_stats))

    return "\n".join(summary_lines)

def main():

    print("\n" + "="*80)
    print("DGA GROUNDWATER RIGHTS DATABASE ANALYSIS")
    print("ALL SPATIAL ASSIGNMENTS FROM SHAPEFILES (NOT Excel columns)")
    print("="*80)
    print(f"Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    create_output_folder(OUTPUT_FOLDER)

    print("\n" + "="*60)
    print("LOADING REFERENCE LAYERS")
    print("="*60)

    reference_gdfs = {}
    for layer_config in REFERENCE_LAYERS:
        gdf, prefix = load_reference_layer(layer_config)
        if gdf is not None:
            reference_gdfs[prefix] = gdf
        else:
            print(f"  WARNING: Failed to load {prefix} layer")

    print(f"\nSuccessfully loaded {len(reference_gdfs)} reference layers: {list(reference_gdfs.keys())}")

    print("\n" + "="*60)
    print("LOADING DGA DATA")
    print("="*60)
    
    df = load_excel_data(EXCEL_PATH)

    print("\n--- Available columns in Excel ---")
    for i, col in enumerate(df.columns):
        print(f"  {i+1}. {col}")

    df, date_quality_stats = process_dates(df, DATE_COL_PRIMARY, DATE_COL_SECONDARY)

    print("\n" + "="*60)
    print("UNIT CONVERSION AND ANOMALY DETECTION")
    print("="*60)

    print("Converting flow rates to L/s...")
    df['Caudal_Ls'] = df.apply(convert_flow_to_ls, axis=1)

    print("Classifying anomalies...")
    df['Anomaly_Status'] = df.apply(classify_anomaly, axis=1)
    df['Anomaly_Severity'] = df.apply(get_anomaly_severity, axis=1)

    print("\n--- Anomaly Classification Summary ---")
    for status, count in df['Anomaly_Status'].value_counts().items():
        pct = 100 * count / len(df)
        print(f"  {status}: {count:,} ({pct:.1f}%)")

    df_valid = df[df['Anomaly_Status'] == 'Valid'].copy()
    df_anomalies = df[df['Anomaly_Status'].isin(['Negative value', 'Unrealistically high value'])].copy()
    df_excluded = df[~df['Anomaly_Status'].isin(['Valid', 'Negative value', 'Unrealistically high value'])].copy()

    print(f"\nValid records for analysis: {len(df_valid):,}")
    print(f"Anomalous records (negative/unrealistic): {len(df_anomalies):,}")
    print(f"Excluded records (non-volumetric/missing): {len(df_excluded):,}")

    print("\n" + "="*60)
    print("CREATING GEODATAFRAMES (WGS84)")
    print("="*60)

    gdf_valid = create_geodataframe(df_valid)
    gdf_anomalies = create_geodataframe(df_anomalies)

    if gdf_valid is not None:
        print(f"  Valid records with coordinates: {len(gdf_valid):,}")
    if gdf_anomalies is not None:
        print(f"  Anomalous records with coordinates: {len(gdf_anomalies):,}")

    print("\n" + "="*60)
    print("PERFORMING SPATIAL JOINS")
    print("="*60)

    for prefix, ref_gdf in reference_gdfs.items():
        gdf_valid = perform_spatial_join(gdf_valid, ref_gdf, prefix)
        if gdf_anomalies is not None:
            gdf_anomalies = perform_spatial_join(gdf_anomalies, ref_gdf, prefix)

    print("\n--- Columns after spatial joins ---")
    spatial_cols = [c for c in gdf_valid.columns if '_Name' in c or '_Code' in c]
    print(f"  Spatial columns added: {spatial_cols}")

    print("\n" + "="*60)
    print("CALCULATING STATISTICS (from spatial joins)")
    print("="*60)

    stats_dict = {}

    for prefix in reference_gdfs.keys():
        name_col = f'{prefix}_Name'
        if name_col in gdf_valid.columns:
            print(f"\nCalculating {prefix} statistics...")
            stats = calculate_statistics(gdf_valid, name_col)
            if len(stats) > 0:
                stats_dict[prefix] = stats
                print(f"  {prefix} units analyzed: {len(stats)}")
            else:
                print(f"  WARNING: No valid statistics for {prefix}")

    temporal_stats, temporal_error = perform_temporal_analysis(gdf_valid, date_quality_stats)

    if temporal_error:
        print(f"WARNING: {temporal_error}")

    print("\n" + "="*60)
    print("SAVING OUTPUTS")
    print("="*60)

    output_valid_path = Path(OUTPUT_FOLDER) / "DGA_Valid_Data_Analysis_SpatialJoin.xlsx"
    print(f"Saving valid data analysis to: {output_valid_path}")
    
    with pd.ExcelWriter(output_valid_path, engine='openpyxl') as writer:
        summary_data = {
            'Metric': [
                'Total Valid Records',
                'Total Anomalous Records',
                'Records with Valid Dates',
                'Total Allocated Flow (L/s)',
                'Mean Flow per Permit (L/s)',
                'Median Flow per Permit (L/s)',
                'Analysis Date',
                'Note'
            ],
            'Value': [
                len(gdf_valid) if gdf_valid is not None else 0,
                len(gdf_anomalies) if gdf_anomalies is not None else 0,
                date_quality_stats['total_with_date'],
                f"{gdf_valid['Caudal_Ls'].sum():,.2f}" if gdf_valid is not None else 0,
                f"{gdf_valid['Caudal_Ls'].mean():.2f}" if gdf_valid is not None else 0,
                f"{gdf_valid['Caudal_Ls'].median():.2f}" if gdf_valid is not None else 0,
                datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'All spatial assignments from shapefiles (not Excel columns)'
            ]
        }
        pd.DataFrame(summary_data).to_excel(writer, sheet_name='Summary', index=False)
        
        for scale_name, stats_df in stats_dict.items():
            if len(stats_df) > 0:
                stats_df.to_excel(writer, sheet_name=f'Stats_{scale_name}', index=False)
        
        if temporal_stats is not None:
            temporal_stats['yearly'].to_excel(writer, sheet_name='Temporal_Yearly', index=False)
            temporal_stats['decade'].to_excel(writer, sheet_name='Temporal_Decade', index=False)
            temporal_stats['period_5yr'].to_excel(writer, sheet_name='Temporal_5Year', index=False)
            if temporal_stats['regional_peaks'] is not None:
                temporal_stats['regional_peaks'].to_excel(writer, sheet_name='Temporal_Regional_Peaks', index=False)

    output_gdf_path = Path(OUTPUT_FOLDER) / "DGA_Data_With_Spatial_Joins.xlsx"
    print(f"Saving data with spatial joins to: {output_gdf_path}")
    
    df_export = pd.DataFrame(gdf_valid.drop(columns='geometry'))
    
    key_cols = ['Código de Expediente', 'Caudal_Ls', 'Ano', 'lat_wgs84', 'lon_wgs84']
    spatial_cols = [c for c in df_export.columns if '_Name' in c or '_Code' in c]
    key_cols.extend(spatial_cols)
    
    available_cols = [c for c in key_cols if c in df_export.columns]
    df_export[available_cols].to_excel(output_gdf_path, index=False)

    output_clean_path = Path(OUTPUT_FOLDER) / "DGA_Data_Clean_With_Spatial_Joins.xlsx"
    print(f"Saving clean dataset to: {output_clean_path}")
    df_export.to_excel(output_clean_path, index=False)

    if temporal_stats is not None:
        output_temporal_path = Path(OUTPUT_FOLDER) / "DGA_Temporal_Analysis.xlsx"
        print(f"Saving temporal analysis to: {output_temporal_path}")
        with pd.ExcelWriter(output_temporal_path, engine='openpyxl') as writer:
            temporal_stats['yearly'].to_excel(writer, sheet_name='Yearly_Statistics', index=False)
            temporal_stats['decade'].to_excel(writer, sheet_name='Decade_Statistics', index=False)
            temporal_stats['period_5yr'].to_excel(writer, sheet_name='5Year_Period_Stats', index=False)
            temporal_stats['top10_permits_years'].to_excel(writer, sheet_name='Top10_Years_Permits', index=False)
            temporal_stats['top10_flow_years'].to_excel(writer, sheet_name='Top10_Years_Flow', index=False)

    print("\nGenerating analysis summary...")
    summary_text = generate_analysis_summary(gdf_valid, gdf_anomalies, stats_dict, temporal_stats)

    output_txt_path = Path(OUTPUT_FOLDER) / "DGA_Analysis_Summary_SpatialJoin.txt"
    print(f"Saving text summary to: {output_txt_path}")
    with open(output_txt_path, 'w', encoding='utf-8') as f:
        f.write(summary_text)

    print("\n" + summary_text)

    print("\n" + "="*80)
    print("ANALYSIS COMPLETE")
    print("="*80)
    print(f"Finished: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"\nOutput files saved to: {OUTPUT_FOLDER}")
    print("\nIMPORTANT: All spatial assignments (Region, Municipality, Basin, SHAC)")
    print("           were determined via spatial joins with reference shapefiles,")
    print("           NOT from existing Excel columns.")

if __name__ == "__main__":
    main()